# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** Ammar Asysyakur
- **Email:** ammarasysyakur723@gmail.com
- **ID Dicoding:** [Input Username]

## Menentukan Pertanyaan Bisnis

- Bagaimana distribusi penjualan berdasarkan kategori produk?
- Bagaimana waktu dan pola pembayaran pelanggan?

## Import Semua Packages/Library yang Digunakan

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Wrangling

### Gathering Data

In [ ]:
orders_df = pd.read_csv("data/orders_dataset.csv")
orders_df.head()

In [ ]:
order_items_df = pd.read_csv("data/order_items_dataset.csv")
order_items_df.head()

In [ ]:
order_payments_df = pd.read_csv("data/order_payments_dataset.csv")
order_payments_df.head()

In [ ]:
product_category_df = pd.read_csv("data/product_category_name_translation.csv")
product_category_df.head()

In [ ]:
products_df = pd.read_csv("data/products_dataset.csv")
products_df.head()

**Insight:**
- 
- Data yang dibutuhkan untuk menjawab pertanyaan pertama adalah data product yaitu nama nama produk, data product_category_transalation yaitu kategori produk beserta terjemahannya kedalam bahasa inggris, dan data order_items yaitu data item pesanan yang meliputi data pesanan, produk, seller, batas pengiriman dan harga
- Sedangkan data yang dibutuhkan untuk pertanyaan kedua ialah data order_payments yaitu data urutan pembayaran, tipe pembayaran, jumlah cicilan dan jumlah pembayaran dan juga data order yaitu pesanan

### Assessing Data

In [ ]:
products_df.info()

In [ ]:
products_df.isna().sum()

In [ ]:
print("Jumlah duplikasi: ", products_df.duplicated().sum())

**Insight:**
- Terlihat beberapa kolom tidak sama dengan jumlahnya dengan product_id maka perlu dilakukan replace data yang missing
- Pengecekan pada duplikasi data data tidak ada
- product_dataset = missing value

In [ ]:
product_category_df.info()

In [ ]:
print("Jumlah duplikasi: ", product_category_df.duplicated().sum())

**Insight:**
- Pada data kategori produk telah dilakukan pengecekan yang menghasilkan 0 untuk missing value dan duplicate data maka data kategori produk aman

In [ ]:
order_items_df.info()

In [ ]:
print("Jumlah duplikasi: ", order_items_df.duplicated().sum())

**Insight:**
- Pada data order items telah dilakukan pengecekan yang menghasilkan 0 untuk missing value dan duplicate data maka data order item aman
- - namun pada beberapa data yang berupa tanggal tipe datanya masih objek

In [ ]:
order_payments_df.info()
# order_payments_df.isna().sum()

In [ ]:
print("Jumlah duplikasi: ", order_payments_df.duplicated().sum())

**Insight:**
- Pada data order payments telah dilakukan pengecekan yang menghasilkan 0 untuk missing value dan duplicate data maka data order payments aman

In [ ]:
orders_df.info()
orders_df.isna().sum()

In [ ]:
print("Jumlah duplikasi: ", orders_df.duplicated().sum())

**Insight:**
- Pada data order  telah dilakukan pengecekan pada data missing value yang hasilnya ada
- sedangkan nilai duplicate tidak ada
- namun pada beberapa data yang berupa tanggal tipe datanya masih objek
- orders = missing value

### Cleaning Data

**Need Cleaning:**
- products_df = missing value
- orders_df = missing value and object to datetime type

In [ ]:
## handling missing value on product_df
products_df.describe()
products_df.isna().sum()

In [ ]:
products_df['product_category_name'].fillna("unknown", inplace=True)

num_cols = ['product_name_lenght', 'product_description_lenght', 
            'product_photos_qty', 'product_weight_g', 
            'product_length_cm', 'product_height_cm', 'product_width_cm']

for col in num_cols:
    median_value = products_df[col].median()
    products_df[col].fillna(median_value, inplace=True)

print(products_df.isna().sum())

In [ ]:
## handling missing value on orders_df
orders_df.describe()
orders_df.isna().sum()

**Insight:**
- untuk data missing di order_approved_at Diisi dengan order_purchase_timestamp karena pesanan pasti dibuat sebelum disetujui.
- untuk data missing di order_delivered_carrier_date Diisi dengan order_approved_at, karena pesanan harus disetujui sebelum dikirim ke kurir.
- untuk data missing di Diisi dengan order_estimated_delivery_date, karena perkiraan pengiriman bisa menjadi acuan jika tidak ada data aktual.
- sebelum data missing  diisi maka dilakukan perubahan tipe data pada beberapa data yang bentuk tanggal masih object

In [68]:
date_cols = [
    "order_purchase_timestamp", 
    "order_approved_at", 
    "order_delivered_carrier_date", 
    "order_delivered_customer_date", 
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders_df[col] = pd.to_datetime(orders_df[col])

orders_df["order_approved_at"] = orders_df["order_approved_at"].fillna(orders_df["order_purchase_timestamp"])
orders_df["order_delivered_carrier_date"] = orders_df["order_delivered_carrier_date"].fillna(orders_df["order_approved_at"])
orders_df["order_delivered_customer_date"] = orders_df["order_delivered_customer_date"].fillna(orders_df["order_estimated_delivery_date"])

In [ ]:
orders_df.isna().sum()

In [ ]:
orders_df.info()

## Exploratory Data Analysis (EDA)

### Explorasi distribusi penjualan berdasarkan kategori produk

In [ ]:
# Merge order_items_df dengan products_df berdasarkan product_id
sales_data = order_items_df.merge(
    products_df[['product_id', 'product_category_name']],
    on='product_id', 
    how='left'
)

# Merge dengan product_category_df untuk mendapatkan kategori dalam bahasa Inggris
sales_data = sales_data.merge(product_category_df, on='product_category_name', how='left')

# Cek jumlah data setelah merging
# print(f"Jumlah total data setelah merging: {sales_data.shape[0]}")
print(sales_data.head())

In [ ]:
# Menghitung Jumlah Produk yang Terjual per Kategori
category_sales = sales_data.groupby('product_category_name_english').agg(
    total_sales=('product_id', 'count')
).reset_index()

# Cek 5 kategori teratas
category_sales = category_sales.sort_values(by='total_sales', ascending=False)
category_sales.head(5)

In [ ]:
# Menghitung Total Pendapatan per Kategori
category_revenue = sales_data.groupby('product_category_name_english').agg(
    total_revenue=('price', 'sum')
).reset_index()

# Urutkan dari yang tertinggi
category_revenue = category_revenue.sort_values(by='total_revenue', ascending=False)
category_revenue.head(5)

**Insight:**
- pada tahap exploratory pada data product, order_items dan product_category ini menggunakan sistem penggabungan untuk memahami distribusi penjualan
- ditemukan pada data diatas beberapa kategori yang penjualannya mendominasi penjualannya per item namun ada beberapa kategori produk yang pendapatannya lebih besar meskipun penjualannya lebih sedikit, yang ini disebabkan oleh harga yang lebih tinggi

### Explorasi pola pembayaran customer

In [ ]:
# Melihat distribusi metode pembayaran
order_payments_df["payment_type"].value_counts()

In [ ]:
# Menggabungkan order_payments_df dengan orders_df
merged_df = order_payments_df.merge(orders_df, on="order_id", how="left")
merged_df.head()

In [85]:
# Konversi ke datetime
merged_df["order_purchase_timestamp"] = pd.to_datetime(merged_df["order_purchase_timestamp"])

# Menambahkan kolom bulan dan tahun
merged_df["order_month"] = merged_df["order_purchase_timestamp"].dt.to_period("M")

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
Bagaimana distribusi penjualan berdasarkan kategori produk?

In [ ]:
# Plot jumlah penjualan per kategori
plt.figure(figsize=(10, 5))
sns.barplot(
    data=category_sales.head(5), 
    x="total_sales", 
    y="product_category_name_english", 
    palette="Blues_r"
)
plt.xlabel("Total Produk Terjual")
plt.ylabel("Kategori Produk")
plt.title("5 Kategori Produk dengan Penjualan Terbanyak")
plt.show()


**Insight:**
- Kategori bed_bath_table memiliki jumlah penjualan tertinggi, diikuti oleh health_beauty dan sports_leisure.

In [ ]:
# Plot total pendapatan per kategori
plt.figure(figsize=(10, 5))
sns.barplot(
    data=category_revenue.head(5), 
    x="total_revenue", 
    y="product_category_name_english", 
    palette="Oranges_r"
)
plt.xlabel("Total Pendapatan (dalam juta)")
plt.ylabel("Kategori Produk")
plt.title("5 Kategori Produk dengan Pendapatan Tertinggi")
plt.show()

**Insight:**
- Kategori watches_gifts memiliki pendapatan tinggi walaupun tidak masuk ke dalam 5 besar kategori dengan jumlah produk terjual.

### Pertanyaan 2:

In [ ]:
Bagaimana waktu dan pola pembayaran pelanggan?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Hitung jumlah transaksi berdasarkan metode pembayaran
payment_counts = order_payments_df["payment_type"].value_counts()

# Visualisasi menggunakan pie chart
plt.figure(figsize=(8, 8))
plt.pie(payment_counts, labels=payment_counts.index, autopct="%1.1f%%", colors=sns.color_palette("pastel"))
plt.title("Distribusi Metode Pembayaran")
plt.show()


**Insight:**
- Metode pembayaran yang paling banyak digunakan adalah credit_card, menunjukkan bahwa mayoritas pelanggan lebih memilih pembayaran non-tunai.
- Metode pembayaran lainnya seperti boleto dan voucher memiliki porsi yang lebih kecil.

In [ ]:
# Melihat tren jumlah transaksi per bulan
merged_df.groupby("order_month")["payment_value"].sum().plot(kind="line", figsize=(12,6), title="Total Pembayaran per Bulan")

**Insight:**
- Transaksi customer pada bulan oktober dan april mengalami pelonjakan yang mungkin bisa disebabkan oleh event tertentu seperti diskon atau yang lainnya

## Conclusion

### 1. Distribusi Penjualan Berdasarkan Kategori Produk
- **Kategori dengan jumlah penjualan terbanyak:** bed_bath_table, health_beauty, sports_leisure, furniture_decor, dan computers_accessories.
- **Kategori dengan pendapatan tertinggi:** health_beauty, watches_gifts, bed_bath_table, sports_leisure, dan computers_accessories.
- **Insight:** Kategori health_beauty memiliki pendapatan tertinggi meskipun bukan kategori dengan jumlah penjualan tertinggi, menunjukkan bahwa harga produk dalam kategori ini lebih tinggi dibandingkan kategori lain.

### 2. Waktu dan Pola Pembayaran Pelanggan
- **Metode pembayaran paling banyak digunakan:** credit_card, diikuti oleh boleto dan voucher.
- **Pola pembayaran per bulan menunjukkan fluktuasi**, dengan beberapa bulan memiliki transaksi lebih tinggi, kemungkinan dipengaruhi oleh momen promosi atau hari besar.